In [1]:
import time

import pysal as ps
from geomet import wkb
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [17]:
#Slurp all the pysal example files into a single sqlite database
import glob
import subprocess
psexamples = glob.glob('/Users/jay/github/pysal/pysal/examples/*/*.shp')
for i, s in enumerate(psexamples):
    cmd = 'ogr2ogr -f SQLite -append /Users/jay/Desktop/psexamples.sqlite {}'.format(s)
    subprocess.check_call(cmd.split())

['/Users/jay/github/pysal/pysal/examples/10740/10740.shp', '/Users/jay/github/pysal/pysal/examples/baltim/baltim.shp', '/Users/jay/github/pysal/pysal/examples/burkitt/burkitt.shp', '/Users/jay/github/pysal/pysal/examples/chicago/Chicago77.shp', '/Users/jay/github/pysal/pysal/examples/columbus/columbus.shp', '/Users/jay/github/pysal/pysal/examples/geodanet/crimes.shp', '/Users/jay/github/pysal/pysal/examples/geodanet/schools.shp', '/Users/jay/github/pysal/pysal/examples/geodanet/streets.shp', '/Users/jay/github/pysal/pysal/examples/juvenile/juvenile.shp', '/Users/jay/github/pysal/pysal/examples/Line/Line.shp', '/Users/jay/github/pysal/pysal/examples/nat/NAT.shp', '/Users/jay/github/pysal/pysal/examples/nat/natregimes.shp', '/Users/jay/github/pysal/pysal/examples/networks/eberly_net.shp', '/Users/jay/github/pysal/pysal/examples/networks/eberly_net_pts_offnetwork.shp', '/Users/jay/github/pysal/pysal/examples/networks/eberly_net_pts_onnetwork.shp', '/Users/jay/github/pysal/pysal/examples/n

NameError: name 'asd' is not defined

In [2]:
class SQLConnection():
    
    def __init__(self, db):
        self.dbname = db
        self.Base = automap_base()
        self._engine = create_engine(self.dbname)
        self.Base.prepare(self._engine, reflect=True)
        
    @property
    def tables(self):
        if not hasattr(self, '_tables'):
            self._tables = {}
            for k in self.Base.classes.keys():
                self._tables[k] = self.Base.classes[k]
        return self._tables

    @property
    def session(self):
        #What happens if the session is externally closed?  Check for None?
        if not hasattr(self, '_session'):
            self._session = Session(self._engine)
        return self._session
    
    def get_gjson(self, tablename, attributes=False, geom_column="GEOMETRY"):
    
        gjson = {
            "type": "FeatureCollection",
            "features": []}

        for row in self.session.query(self.tables[tablename]):
            feat = {"type": "Feature",
                    "geometry": {},
                    "properties":{}}

            feat["geometry"] = wkb.loads(row.GEOMETRY)
            attributes = row.__dict__
            attributes.pop('_sa_instance_state', None)
            attributes.pop(geom_column, None)
            feat["properties"] = attributes
            gjson["features"].append(feat)

        return gjson

In [3]:
conn = SQLConnection("sqlite:////Users/jay/Desktop/psexamples.sqlite")

In [4]:
conn.tables.keys()

['burkitt',
 'stl_hom',
 'juvenile',
 'point',
 'sacramentot2',
 'taz',
 'eberly_net',
 'polygon',
 'eberly_net_pts_offnetwork',
 'natregimes',
 'lattice10x10',
 'nat',
 'sids2',
 '10740',
 'columbus',
 'street_net_pts',
 'virginia',
 'nonplanarsegments',
 'line',
 'rook31',
 'us48',
 'baltim',
 'chicago77',
 'eberly_net_pts_onnetwork',
 'south']

In [27]:
t1 = time.time()
gjson = conn.get_gjson('nat')
t2 = time.time()
print t2 - t1
t3 = time.time()
w = contiguity_from_geojson(gjson, wtype='rook')
t4 = time.time()
print t4 - t3
print '------------'
print t4 - t1

2.06201100349
3085
setting bounding box for FeatureCollection
3.89124917984
------------
5.95346713066


In [30]:
#%load_ext line_profiler
%lprun -f contiguity_from_geojson contiguity_from_geojson(gjson, wtype='rook')

setting bounding box for FeatureCollection



In [14]:
t1 = time.time()
ps_w = ps.queen_from_shapefile('/Users/jay/github/pysal/pysal/examples/nat/nat.shp')
t2 = time.time()
print t2 - t1

1.27663397789


In [32]:
ps_w = ps.queen_from_shapefile(ps.examples.get_path('NAT.shp'))

Is the performance issue caused by the conversion from WKB to GeoJSON, database access, or the generalized builder because it is iterating over GeoJSON?

In [60]:
t1 = time.time()
gjson = conn.get_gjson('columbus')
w = contiguity_from_geojson(gjson, wtype='queen')
t2 = time.time()
print t2 - t1

setting bounding box for FeatureCollection
0.128070831299


In [18]:
t1 = time.time()
gjson = conn.get_gjson('columbus')
w = contiguity_from_geojson(gjson, wtype='rook')
t2 = time.time()
print t2 - t1

setting bounding box for FeatureCollection
0.149102926254


In [29]:
#Serge's code from: https://github.com/sjsrey/pysal/blob/wconstructor/pysal/weights/json.ipynb
def parse_geojson(resource):
    return json.loads(resource)

def contiguity_from_geojson(gj_input, wtype='rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        print "wtype must be 'QUEEN' or 'ROOK'"
    WTYPE =["QUEEN", 'ROOK'].index(WTYPE)+1
    collection = gj_input
    n = len(collection)
    collection_type = collection['features'][0]['geometry']['type']
    if collection_type in ['Polygon', 'MultiPolygon']:
        polys = []
        ids = []
        i = 0
        pappend = polys.append
        makeshape = ps.cg.asShape
        ids = xrange(len(collection['features']))
        for feature in collection['features']:
            pappend(makeshape(feature['geometry']))
            #ids.append(i)
            #i +=1
        pcollection = ps.cg.shapes.PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=WTYPE).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"

In [31]:
w = contiguity_from_geojson(gjson, wtype='queen')

setting bounding box for FeatureCollection


In [33]:
w_ps = ps.queen_from_shapefile(ps.examples.get_path('columbus.shp'))

In [46]:
for k, v in w.neighbors.iteritems():
    assert(list(v) == w_ps.neighbors[k])

In [30]:
import numpy as np
import pysal.cg.shapes as shp

class PolygonCollection:
    def __init__(self, polygons, bbox=None):
        """
        Parameters
        ==========
        polygons: dict
                  key is polygon Id, value is PySAL Polygon object
        bbox: list (optional)
              [left, lower, right, upper]
        Notes
        =====
        bbox is supported in geojson specification at both the feature and feature collection level. However, not all GeoJSON writers generate the bbox at the feature collection level. 
        In those cases, the bbox property will be set on initial access.
        """
              
        self.type = shp.Polygon
        self.n = len(polygons)
        self.polygons = polygons
        if bbox is None:
            self._bbox = None
        else:
            self._bbox = bbox
            
    @property
    def bbox(self):
        if self._bbox is None:
            print 'setting bounding box for FeatureCollection'
            bboxes = np.array([self.polygons[p].bbox for p in self.polygons])
            mins = bboxes.min(axis=0)
            maxs = bboxes.max(axis=0)
            self._bbox = [ mins[0], mins[1], maxs[2], maxs[3] ]
        return self._bbox
        
    
    def __getitem__(self, index):
        return self.polygons[index]
    
ps.cg.shapes.PolygonCollection = PolygonCollection

In [ ]:
class SqlConnection(lite.Connection):
    def __init__(self, *args):
        super(SqlConnection, self).__init__(*args)
        self.cur = self.cursor()

    def execute(self, query):
        self.cur.execute(query)
    
    @property
    def tables(self):
        if not hasattr(self, '_tables'):
            query = 'SELECT name FROM sqlite_master WHERE type="table" ORDER BY name' 
            self.execute(query)
            self._tables = self.cur.fetchall()
        return self._tables
    
    def get_columns(self, table):
        query = "PRAGMA table_info('{}')".format(table)
        self.execute(query)
        return [c[:3] for c in self.cur]
    
    def get_table(self, table, columns=['*'], where=None):
        columns = ','.join(columns)
        query = 'SELECT {} FROM {}'.format(columns, table)
        if where:
            query = query + ' {}'.format(where)
        self.execute(query)
        column_names = [description[0].lower() for description in self.cur.description]
        return column_names, self.cur

class Table():
    
    def __init__(self, db, tablename):
        self.tablename = tablename
        self.db = db
        
    @property
    def columns(self):
        if not hasattr('self', _columns):
            query = "PRAGMA table_info('{}')".format(self.tablename)
            self.db.execute(query)
            